In [1]:
from keras.models import Model
from keras.layers import *
from keras import backend as K
from keras.optimizers import Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)


def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)


def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [3]:
def gaussian2D(shape, mean, cov):
    '''
    add gaussian to source image
    params:
        shape - shape of the output
        mean - array-like tuple of x_mean, y_mean - relative positions
        cov - 2x2 covariance matrix
        magnitude - maximal intensity of gaussian
    '''
    X, Y = np.indices(shape)
    X = (X + 1).astype(np.float) / (shape[0] + 1)
    Y = (Y + 1).astype(np.float) / (shape[1] + 1)
    r = np.stack([X, Y], -1) - mean
    exp = np.einsum('ijk,kl,ijl->ij', r, np.linalg.inv(cov), r)
    exp = np.exp(-0.5 * exp)
    return exp

def random_small_rotation():
    angle = np.random.normal(0, scale=0.1)
    matrix = np.array([
        [np.cos(angle), np.sin(angle)],
        [-1.0 * np.sin(angle), np.cos(angle)]
    ])
    return matrix

def add_gaussian_on_image(image, magnitude, mean, radius, cov):        
    new_image = image.copy()
    gauss = gaussian2D(image.shape, mean, radius * cov)
    new_image += gauss * magnitude
    return new_image

def gen_random_images():

    # Background
    dark_color = np.random.randint(50, 150)
    light_color = np.random.randint(1, 255 - dark_color)
    
    img = get_random_background() * dark_color
    t_vec = np.random.random(size=2)
    t_vec = t_vec / np.linalg.norm(t_vec)
    
    images = []
    masks = []
    
    
    point = np.random.random(size=2)
    radius = np.random.uniform(0.0001, 0.0025)
    cov = np.eye(2) + 0.5 * np.random.uniform(-1, 1, size=(2, 2))
    
    for _ in range(N_IMAGES):
        generated_img = add_gaussian_on_image(img, light_color, point, radius, cov)
        images.append(generated_img)
        
        mask = add_gaussian_on_image(np.zeros_like(img), 1.0, point, radius, cov) > 0.5
        mask = mask.astype('float32')
        masks.append(mask)
        
        t_vec = random_small_rotation().dot(t_vec)
        distance = 0.1 * np.random.normal(1, scale=0.1)
        point += t_vec * distance
        cov = random_small_rotation().dot(cov)
        radius *= np.random.uniform(0.7, 0.9)
        
    # White noise
    noise = np.random.randint(0, 200, size=image_size)
    p_noise = np.random.random(size=image_size)
    th = np.random.uniform(0, 0.1)
    for img in images:
#         pass
        img[p_noise < th] = noise[p_noise < th]
#         new_images.append(np.expand_dims(img[i], -1
    
    return np.stack(images, axis=-1), np.stack(masks, axis=-1)

In [4]:
def double_conv_layer(x, size, dropout, batch_norm):
    axis = 3
    conv = Conv2D(size, (3, 3), padding='same')(x)
    if batch_norm is True:
        conv = BatchNormalization(axis=axis)(conv)
    conv = Activation('relu')(conv)
    conv = Conv2D(size, (3, 3), padding='same')(conv)
    if batch_norm is True:
        conv = BatchNormalization(axis=axis)(conv)
    conv = Activation('relu')(conv)
    if dropout > 0:
        conv = Dropout(dropout)(conv)
    return conv

In [5]:
def ZF_UNET_224(dropout_val=0.0, batch_norm=True):
    inputs = Input((512, 512, 3))
    axis = 3
    filters = 16

    conv_224 = double_conv_layer(inputs, filters, dropout_val, batch_norm)
    pool_112 = MaxPooling2D(pool_size=(2, 2))(conv_224)

    conv_112 = double_conv_layer(pool_112, 2*filters, dropout_val, batch_norm)
    pool_56 = MaxPooling2D(pool_size=(2, 2))(conv_112)

    conv_56 = double_conv_layer(pool_56, 4*filters, dropout_val, batch_norm)
    pool_28 = MaxPooling2D(pool_size=(2, 2))(conv_56)

    conv_28 = double_conv_layer(pool_28, 8*filters, dropout_val, batch_norm)
    pool_14 = MaxPooling2D(pool_size=(2, 2))(conv_28)

    conv_14 = double_conv_layer(pool_14, 16*filters, dropout_val, batch_norm)
    pool_7 = MaxPooling2D(pool_size=(2, 2))(conv_14)

    conv_7 = double_conv_layer(pool_7, 32*filters, dropout_val, batch_norm)

    up_14 = concatenate([UpSampling2D(size=(2, 2))(conv_7), conv_14], axis=axis)
    up_conv_14 = double_conv_layer(up_14, 16*filters, dropout_val, batch_norm)

    up_28 = concatenate([UpSampling2D(size=(2, 2))(up_conv_14), conv_28], axis=axis)
    up_conv_28 = double_conv_layer(up_28, 8*filters, dropout_val, batch_norm)

    up_56 = concatenate([UpSampling2D(size=(2, 2))(up_conv_28), conv_56], axis=axis)
    up_conv_56 = double_conv_layer(up_56, 4*filters, dropout_val, batch_norm)

    up_112 = concatenate([UpSampling2D(size=(2, 2))(up_conv_56), conv_112], axis=axis)
    up_conv_112 = double_conv_layer(up_112, 2*filters, dropout_val, batch_norm)

    up_224 = concatenate([UpSampling2D(size=(2, 2))(up_conv_112), conv_224], axis=axis)
    up_conv_224 = double_conv_layer(up_224, filters, 0, batch_norm)

    conv_final = Conv2D(3, (1, 1))(up_conv_224)
    conv_final = BatchNormalization(axis=axis)(conv_final)
    conv_final = Activation('sigmoid')(conv_final)

    model = Model(inputs, conv_final, name="ZF_UNET_224")
    return model